## Imports

In [1]:
import os, argparse, numpy as np
import matplotlib.pyplot as plt 
import illustris_python as il
print("Imports Done")

# A useful print fucntion I often use for printing
def tabprint( printme, start = '\t - ', end = '\n' ):
    print( start + printme, end = end )

Imports Done


---
## Command Line Arguments

This is written in JupyterLab, but will eventually be compiled and ran in python for faster results.  This will define the possible input command line arguements

In [2]:
# This argument decides if code is in python or jupyter.
buildEnv = False

# Define argument parser function 
def initParser():
    
    parser = argparse.ArgumentParser()
    
    parser.add_argument( '-s', '--simDir', default = '../sims.TNG/TNG100-1/output/',  type=str, \
                        help="Base directory for a single simulation on the IllustrisTNG servers.")   
    
    parser.add_argument( '-n', '--simName', default = 'TNG100-1',  type=str, \
                        help="Name for the simulation being worked on.")
    
    parser.add_argument( '-o', '--overwrite', default = False,  type=bool, \
                        help="Overwrite output files?  If false, will check if output file exists before beginning time-consuming tasks.")
    
    parser.add_argument( '-t', '--trim', default = -1,  type=int, \
                        help="Default number of subhalos to consider, sorted by highest mass first.")
    
    parser.add_argument( '-f', '--function', default = 'None', type=str, \
                        help="Default function program will be executing.")

    return parser

parser = initParser()
print("Args: Defined")

Args: Defined


## To Python? Or to JupyterLab? 
This will establish if this is being run in a JupyterLab environment or from Command Line in Python. 

NOTE:  If you're running this in Jupyter, modify the `cmdStr` below to whatever variable you need.

In [3]:
# Am I in a jupyter notebook?
try:
    
    # This command is NOT available in a python script
    get_ipython().__class__.__name__
    buildEnv = True
    print ("In Building Environment")
    
    # Command Line Arguments
    cmdStr  = 'python3 basic-working.py'
    cmdStr += ' --simDir ../sims.TNG/TNG100-1/output/'
    cmdStr += ' --simName TNG100-1'
    
    # Read string as if command line
    print( "CMD Line: \n\t$:", cmdStr)
    
    # This function doesn't like the 'python3 file.py' part.
    args = parser.parse_args(cmdStr.split()[2:])

# Or am I in a python script?
except:
    
    # Read CMD arguments
    args = parser.parse_args()
    

print( "Args: Read")
print( args )

In Building Environment
CMD Line: 
	$: python3 basic-working.py --simDir ../sims.TNG/TNG100-1/output/ --simName TNG100-1
Args: Read
Namespace(simDir='../sims.TNG/TNG100-1/output/', simName='TNG100-1', overwrite=False, trim=-1, function='None')


In [4]:
if buildEnv: 
    # Location of one simulation
    print("Is this locational valid?")
    print( f"Simulation data: {os.path.exists( args.simDir )} - {args.simDir}" )

Is this locational valid?
Simulation data: True - ../sims.TNG/TNG100-1/output/


---
# Halos and SubHalos
Within the simulation, Halos are the largest set of objects that are gravitationally bound to each other, I like to think of them as galaxy clusters.  Subhalos are also gravitationally bound objects but more dense.  I like to think of them as galaxies, globular clusters, etc.

___
# Centrals and Satellites
Halo's often have a singular central galaxy that's the largest, with smaller subhalos orbiting it called satellites.  For our purposes, we will be focusing on the central galaxies because they are the largest and have most likley undergone the most mergers. Let's get a list of subhalo IDs for the central galaxies.

Note 1: The ID is an index for another list.

Note 2: The Halos are ordered by largest mass first.

Note 3: The jump between subhalo id's between halos tells us how many subhalos the halo has. 

In [5]:
def getCentralIds( args, snapshot = 99, trim=-1 ):
    
    cLoc = f'{args.simName}-central-galaxies.txt'

    # If already obtained, read from file
    if os.path.exists( cLoc ) and not args.overwrite:
        print(f"Reading Central Galaxy ID file: {cLoc}")
        central_ids = np.loadtxt( cLoc, skiprows=1, dtype=int )

    # Else, use IllustricTNG to get central IDs
    else:
        print(f"Getting Central SubHalo IDs for sim: {args.simName}")
        
        # The GroupFirstSub is the subhalo id for the largest subhalo in a halo.  
        GroupFirstSub = il.groupcat.loadHalos( args.simDir, snapshot, fields=['GroupFirstSub'])
        
        # Filter out groups that contain no subhalos.
        w = np.where(GroupFirstSub >= 0) # value of -1 indicates no subhalo in this group
        central_ids = GroupFirstSub[w]
        
        print( f"Writing Central IDs to file: {cLoc}" )

        cFile = open( cLoc, 'w' )
        cFile.write("SubHaloID \n")
        for c in central_ids:
            cFile.write(f"{c} \n")
        cFile.close()

        tabprint("Writing Complete")
    
    # Trim some from the bottom, after saving all the 
    if trim != -1 and central_ids.shape[0] > trim:
        central_ids = central_ids[0:trim]
        
    return central_ids


if buildEnv and True: 

    central_ids = getCentralIds( args, trim=args.trim )

    print( f'Galactic Cluster Count: {central_ids.shape}' )
    print( f'Top 10 IDs: {central_ids[0:10]}' )
    
elif args.function == 'central-ids':
        central_ids = getCentralIds( args, trim=args.trim )

Reading Central Galaxy ID file: TNG100-1-central-galaxies.txt
Galactic Cluster Count: (3430706,)
Top 10 IDs: [    0 17185 31342 41582 52618 60731 69507 76086 83280 88663]


# Counting Significant Merger events
We are specifically looking for major and minor mergers, where the secondary galaxy has a significant fraction of mass compared to the primary galaxy.  Let's define a ratio and look for mergers where the secondary galaxy needs at least that much mass in comparison to the primary galaxy.  

Note: The following built in function tells us *how many* they've undergone, but not *when* they've undergone merger events.  But this could still be useful. Since we have 3 Million central galaxies to choose from, it might be useful to find galaxies that have undergone several mergers.  

In [9]:
def getMergerCount( central_ids, args, ratio=1/10 ):
    
    # Check if file exists
    if args.trim == -1:
        fLoc = f'{args.simName}-merger-count.txt'
    else:
        fLoc = f'{args.simName}-merger-count-{args.trim}.txt'

    # If already obtained, read from file
    if os.path.exists( fLoc ) and not args.overwrite:
        print(f"Reading Merger Count file: {fLoc}")
        merger_count_merged = np.loadtxt( fLoc, skiprows=1, dtype=int )

    # Else, use IllustricTNG to get Merger Count
    else:    
    
        # the following fields are required for the walk and the mass ratio analysis
        fields = ['SubhaloID','NextProgenitorID','MainLeafProgenitorID','FirstProgenitorID','SubhaloMassType']
        
        n = central_ids.shape[0]
        merger_count = np.zeros( n, dtype=int )

        for i, id in enumerate( central_ids ):

            # Get merger tree
            tree = il.sublink.loadTree( args.simDir, 99, id, fields=fields )

            # Pass merger tree into function to count mergers above ratio threshold
            merger_count[i] = il.sublink.numMergers( tree, minMassRatio=ratio )

            print( f"\n - {i} - {id} - {merger_count[i]}", end='\r' )
        
        print( f"\nWriting Merger Count to file: {fLoc}" )

        oFile = open( fLoc, 'w' )
        oFile.write(f"SubHaloID Merger-Count-{ratio}\n")
        for i in range( n ):
            oFile.write(f"{central_ids[i]} {merger_count[i]}\n")
        oFile.close()

        tabprint("Writing Complete")
    
    
    

if buildEnv and True:
    args.trim = 10
    central_ids = getCentralIds( args, trim=args.trim )    
    getMergerCount( central_ids, args )

# In python script
elif args.function == 'merger-count':    
    central_ids = getCentralIds( args, trim=args.trim )
    getMergerCount( central_ids, args )

Reading Central Galaxy ID file: TNG100-1-central-galaxies.txt

 - 0 - 0 - 10
 - 1 - 17185 - 7
 - 2 - 31342 - 12
 - 3 - 41582 - 11
 - 4 - 52618 - 5
 - 5 - 60731 - 4
 - 6 - 69507 - 11
 - 7 - 76086 - 8
 - 8 - 83280 - 11
 - 9 - 88663 - 8
Writing Merger Count to file: TNG100-1-merger-count-10.txt
	 - Writing Complete


## Understanding Trees as lists

So I had a crash course in how trees can be represented as lists. Here are some of my key findings while trying to udnerstand the Tree. 

- This is a Tree, flattened into a list. 
- Index 0 is the root of the tree.  The original SubHaloID I fed into the loadTree function. 
- A single galaxy will have multiple SubHaloIDs, a seperate ID for each snapshot.
- FirstProgenitorID: ID for this galaxy during the previous snapshot.  -1 if child node.
- NextProgenitorID: ID of a secondary galaxy/subhalo if merging occured.  -1 if no secondary galaxy.
    - NOTE: If NextProgenitorID has a NextProgenitorID, then 3 or more subhalos were involved.  
- SubhaloMassType, SubhaloMass:  These are NOT sublink.Tree fields.  This implies the tree used the SubhaloID and queried the Group Catalog for additional subhalo info.  Helpful.
- SubhaloIDRaw = SnapNum*10^12 + SubfindID

# Central Masses over time
It is theorized that galaxies typically gain most of their mass through collisions with other galaxies.  Let's plot the mass growing over the simulation snapshots.  Notice the mass growth is on a log scale. Also notice sharp dips in some galaxies.  These dips are likley galaxies currently undergoing merger events, where the missing mass was labeled as belonging to the other galaxy before final collision.  

In [13]:
if buildEnv and True:
    
    args.trim = -1
    central_ids = getCentralIds( args )
    
    # Variables
    final_snapshot = 99
    seed_subhalo_id = central_ids[100]
    fields = ['SubhaloID','NextProgenitorID','MainLeafProgenitorID','FirstProgenitorID',\
              'SubhaloMassType','SubhaloIDRaw','SubhaloMass']

    # Query for the tree of the subhalo
    tree = il.sublink.loadTree( args.simDir, final_snapshot, seed_subhalo_id, fields=fields) 

    # Create a dictionary to map Subhalo IDs to their index in the list
    subhalo_index = {subhalo_id: index for index, subhalo_id in enumerate(tree['SubhaloID'])} 

    # Loop through tree. identify mergers is mass fractions greater than ratio.  

    d = 0 # Prespecified depth
    stop = 10
    pi = 0  # Primary subhalo index
    si = -1 # Secondary subhalo index
    while d < stop:
        print( 'Step:', d )
        tabprint( f"SubhaloID: {tree['SubhaloID'][pi]}" )
        tabprint( f"FirstProgenitorID: {tree['FirstProgenitorID'][pi]}" )
        tabprint( f"NextProgenitorID: {tree['NextProgenitorID'][pi]}" )

        # if present, potential merger
        si = subhalo_index.get( tree['NextProgenitorID'][pi], -1 )

        if si != -1:
            print("MERGER!!!!")
            pm = tree['SubhaloMass'][pi]
            sm = tree['SubhaloMass'][si]
            tabprint( f"{sm/(pm+sm)} - {pm} - {sm}")

        pi = subhalo_index[tree['FirstProgenitorID'][pi]]
        d += 1


Reading Central Galaxy ID file: TNG100-1-central-galaxies.txt
Step: 0
	 - SubhaloID: 50000002800000000
	 - FirstProgenitorID: 50000002800000001
	 - NextProgenitorID: -1
Step: 1
	 - SubhaloID: 50000002800000001
	 - FirstProgenitorID: 50000002800000002
	 - NextProgenitorID: 50000002800035014
MERGER!!!!
	 - 0.0002408809377811849 - 1001.5435180664062 - 0.24131087958812714
Step: 2
	 - SubhaloID: 50000002800000002
	 - FirstProgenitorID: 50000002800000003
	 - NextProgenitorID: 50000002800034424
MERGER!!!!
	 - 1.0867302080441732e-05 - 976.9623413085938 - 0.010617060586810112
Step: 3
	 - SubhaloID: 50000002800000003
	 - FirstProgenitorID: 50000002800000004
	 - NextProgenitorID: 50000002800026069
MERGER!!!!
	 - 0.49958372116088867 - 985.0790405273438 - 983.4401245117188
Step: 4
	 - SubhaloID: 50000002800000004
	 - FirstProgenitorID: 50000002800000005
	 - NextProgenitorID: 50000002800024516
MERGER!!!!
	 - 0.003453451907262206 - 3.3554980754852295 - 0.011628208681941032
Step: 5
	 - SubhaloID: 5000

In [68]:
if buildEnv and False:
    import matplotlib.pyplot as plt 
    fig = plt.figure(figsize=(10,8))
    ax = fig.add_subplot(111)

    fields = ['SubhaloMass','SubfindID','SnapNum']
    start = 99
    for i in range(start, start+5):
        tree = il.sublink.loadTree( args.simDir, 99, central_ids[i], fields=fields, onlyMPB=True)
        mass_msun = tree['SubhaloMass'] * 1e10 / 0.6774
        ax.plot(tree['SnapNum'], mass_msun, '-', label='sub %d' % central_ids[i])

    ax.set_yscale('log')
    ax.set_xlabel('Snapshot Number')
    ax.set_ylabel('Total Subhalo Mass [M$_\odot$]')
    ax.legend();

In [69]:
if buildEnv and True:
    # Let's experiment

    # Load a tree for a subhalo and see what info we get
    fields = ['SubhaloID','NextProgenitorID','MainLeafProgenitorID','FirstProgenitorID',\
              'SubhaloMassType','SubhaloIDRaw','SubhaloMass',]
    tree = il.sublink.loadTree( args.simDir, 99, central_ids[100], fields=fields) 

    print( "Let's print what type of into the tree contains")
    print( "Seed SubHaloID:", central_ids[100] )
    print( 'Type:', type(tree))
    print( 'Keys:', tree.keys() )
    print( 'Count:', tree['count'] )  # What is the 'count' counting? 

    for k in tree.keys():
        try:
            tabprint( f"{k} - {type(tree[k])} - {tree[k].shape} " )
            tabprint( f"{tree[k][:4]}" )
        except:
            tabprint( f"{k}: {type(tree[k])}" )


    for i in range( 2 ):    
        print( i )
        for k in tree.keys():
            try:
                tabprint( f"{k} - {tree[k][i]}" )
            except:
                pass


In [65]:
if buildEnv and False:
    import illustris_python as il

    def find_major_mergers(subhalo_id, cutoff_ratio=0.1, sim_dir= '../sims.TNG/TNG100-1/output/', final_snapshot=99):
        """
        Find major mergers for a given subhalo ID based on a specified mass ratio.

        Parameters:
        sim_dir (str): Directory of the simulation data.
        final_snapshot (int): The final snapshot number to consider.
        subhalo_id (int): The ID of the subhalo to analyze.
        ratio (float): The mass ratio threshold to define a major merger.

        Returns:
        list of tuples: Each tuple contains (subhalo_id, merger_ratio).
        """
        # Fields to load from the merger tree
        fields = ['SubhaloID', 'NextProgenitorID', 'FirstProgenitorID', 'SubhaloMass']
        fields = ['SubhaloID','NextProgenitorID','MainLeafProgenitorID','FirstProgenitorID',\
              'SubhaloMassType','SubhaloIDRaw','SubhaloMass']

        # Load the merger tree
        tree = il.sublink.loadTree(sim_dir, final_snapshot, subhalo_id, fields=fields)

        # Create a dictionary to map Subhalo IDs to their index in the list
        subhalo_index = {id: index for index, id in enumerate(tree['SubhaloID'])}

        # List to store major mergers
        major_mergers = []

        # Start with the seed subhalo
        current_index = 0

        # Traverse the tree
        while current_index != -1:
            current_subhalo_id = tree['SubhaloID'][current_index]
            next_progenitor_id = tree['NextProgenitorID'][current_index]

            # Check for a potential merger
            next_progenitor_index = subhalo_index.get(next_progenitor_id, -1)
            if next_progenitor_index != -1:
                primary_mass = tree['SubhaloMass'][current_index]
                secondary_mass = tree['SubhaloMass'][next_progenitor_index]
                merger_ratio = secondary_mass / (primary_mass + secondary_mass)

                if merger_ratio >= ratio:
                    major_mergers.append((current_subhalo_id, merger_ratio))

            # Move to the first progenitor
            current_index = subhalo_index.get(tree['FirstProgenitorID'][current_index], -1)

        return major_mergers

    # Example usage
    sim_dir = '../sims.TNG/TNG100-1/output/'
    final_snapshot = 99
    subhalo_id = 101  # Replace with actual subhalo ID
    ratio = 0.1  # Replace with desired ratio

    major_mergers = find_major_mergers(sim_dir, final_snapshot, subhalo_id, ratio)
    print(major_mergers)